In [ ]:
""" Для валидации поделим тренировочный набор на 75% обучающего набора,
25% тестового набора. На 75% обучающего набора правильность ~0.93. 
На остальных 25% accuracy ~0.87. С % категории для тестового набора определилсь верно.

Выходной файл: 'result_for_test.csv' с двумя колонками item_id, category_id

По иерархии для категорий:
                  train_score | accuracy_score 
низшая иерархия |    0.97          0.949
средняя иерархия|    0.96          0.928
высшая иерархия |    0.93          0.867

Хар-ки машины: Core i5-4460S 2.9GHz 8GB RAM (время обработки)

В train_test_split параметр shuffle=False, чтобы обучающие и тестовые данные были 
одинакового состава при анализе категорий по иерархии.
"""

import numpy as np
import re
import time
print('helloworld')
import pandas as pd
# импортируем pymorphy2 для морфологического разбора русских слов
import pymorphy2

In [ ]:
# загрузим тренировочный набор
data_frame = pd.read_csv('train.csv')
#data_frame.info() # пустых значений в датафрейме нет

In [ ]:
# создаем объект MorphAnalyzer для морфологического разбора слова
morph = pymorphy2.MorphAnalyzer()

def clean_corpus(document, num=3):
    """ Принимает документ(объявление). Далее очистка и морфологический 
    разбор.
    num = 3 для обработки 'описания(description)' объявления
    num = 2 обработка 'заголовка(title)', необходимо оставить также числа
    """
    if num == 3:
        document = re.sub(
            r'[\W|?|$|.|!|,|\d|(|)|{|}|*|+|%|#|@|^|"|\'|/|\t|;|:|_|-]', 
            r' ', 
            document)
    else:
        # оставить числа в названии объявления(важно)
        document = re.sub(
            r'[\W|?|$|.|!|,|(|)|{|}|*|+|%|#|@|^|"|\'|/|\t|;|:|_|-]', 
            r' ', 
            document)
        
    # для большей информации по граммемам смотри документацию
    # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
    # постараемся оставить только существительные и прилагательные
    # исключив остальные части речи уменьшим шум на данных
    
    document = [word.lower() for word in document.split() if len(word)>num]
    clean_corpus = []
    # морфологический разбор слова в документе
    for word in document:
        m = morph.parse(word.replace('.',''))
        if len(m) != 0:
            clean_word = m[0]
            try:
                if clean_word.tag.POS not in ('NUMR', 'PRCL', 'INTJ', 'PREP', 
                                              'ADVB', 'ADJS', 'CONJ', 'PRED', 
                                              'VERB', 'INFN', 'PRTF', 'GRND', 
                                              'COMP'):
                    clean_corpus.append(clean_word.normal_form)
            except TypeError:
                clean_corpus.append(clean_word.word)
    # удалим дубликаты
    clean_corpus = list(set(clean_corpus))
    return ' '.join(clean_corpus)

In [ ]:
start_time = time.time()
# применим функцию clean_corpus к столбцам description и title датафрейма
data_frame['description'] = data_frame['description'].apply(clean_corpus)
data_frame['title'] = data_frame['title'].apply(clean_corpus, num=2)
print("--- %s min ---" % round(((time.time() - start_time)/60),3))
# --- 45.799 min ---

In [ ]:
def combine_columns(df, new_title, first_col, second_col):
    """ Принимает датафрейм, название для новой колонки,
    две колонки для объядинения. Возвращает датафрейм с новой
    колонкой.
    """
    df[new_title] = df[first_col].astype(str).str.cat(df[second_col].astype(str), sep=' ')
    return df

def clean_data(document):
    """ Принимает документ и удаляет дубликаты слов.
    Вовзращает строку.
    """
    return ' '.join(list(set(document.split())))

In [ ]:
# объединим заголовок, описание и цену в одну колонку
combined_data_frame = combine_columns(data_frame, 'title_desc', 'title', 'description')
combined_data_frame = combine_columns(combined_data_frame, 'title_desc_price', 'title_desc', 'price')

In [ ]:
# после объединения необходимо повторно удалить дубликаты из столбца
# так как заголовок вместе с описанием могут содержать их
combined_data_frame['title_desc_price'] = combined_data_frame['title_desc_price'].apply(clean_data)

In [ ]:
# чтобы определить правильность на тестовом наборе, попробуем определить
# правильность на 75% тренировочного наборе с соответствующими метками
# оставшиеся 25% данных с метками объявляются тестовым набором
# разделим данные combined_data_frame на 75% и 25%
combined_data_frame = data_frame
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(combined_data_frame['title_desc_price'], 
                                                    combined_data_frame['category_id'], 
                                                    shuffle=False)

In [ ]:
# выделим признаки и обучим модель на 75% тренировочного набора
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

start_time = time.time()
# выделение признаков и tf_idf
# создадим экземпляр класса TfidfVectorizer и подгоняем fit можель к нашим данным
vect = TfidfVectorizer(min_df=5, norm=None).fit(X_train)
# получим представление "мешок слов"
X_data_vect = vect.transform(X_train)
# обучение модели
# после перекрестной проверки параметров лучшим параметром регуляризации оказалось C=0.01
model = LogisticRegression(C=0.01).fit(X_data_vect, y_train)
score = model.score(X_data_vect, y_train)
end_of_time = round(((time.time() - start_time)/60),3)

print("--- %s min ---" % end_of_time)
print("Правильность на 75% тренировочного набора: {:.2f}".format(score))
# --- 12.127 min ---
# Правильность на 75% тренировочного набора: 0.93

In [ ]:
# применим модель на 25% тестового набора и сравним
# предсказанные метки с истинными
from sklearn.metrics import accuracy_score
y_predictions = model.predict(vect.transform(X_test))
print("Accuracy: {:.3f}".format(accuracy_score(y_predictions, y_test)))
# Accuracy: 0.862

In [ ]:
# прочитаем файл с категориями для анализа по иерархии
category_frame = pd.read_csv('category.csv')
category_frame['name'] = category_frame['name'].str.lower()

In [ ]:
# заменим category_id в тренировочном наборе согласно иерархии
# низшая иерархия(пример): Бытовая электроника, для дома и дачи и т.д.
def create_dict(num):
    """ Вернем новую серию для иерархии.
    """
    dictionary = {}
    for name, cat_id in zip(category_frame['name'], category_frame['category_id']):
        name = name.replace(',','').split('|')
        try:
            dictionary[cat_id] = name[num]
        except IndexError:
            dictionary[cat_id] = name[num-1]
    
    copy_combined_data_frame = combined_data_frame
    new_category = {}
    new_category['category_id'] = copy_combined_data_frame['category_id']
    new_category = new_category['category_id'].map(dictionary)
    return new_category

In [ ]:
from sklearn.preprocessing import LabelEncoder
# закодируем новые категории
class_label = LabelEncoder()
# веренеи для каждой категории серию числовых меток согласно иерархии
low_level = class_label.fit_transform(create_dict(0))# низшая иерархия: 
mid_level = class_label.fit_transform(create_dict(1))# средняя иерархия: 
hight_level = class_label.fit_transform(create_dict(2))# высшая иерархия:

In [ ]:
# обучим тренировочный набор на данных с категорией по низшкей иерархии
X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(combined_data_frame['title_desc_price'], 
                                                                    low_level,
                                                                    shuffle=False)
# обучение модели
# так как тренировочный набор сохранаятся(shuffle=False) возьмем vect с первой инициализацией
model_low = LogisticRegression(C=0.01).fit(X_data_vect, y_train_low)
score_low = model_low.score(X_data_vect, y_train_low)
y_pred_low = model_low.predict(vect.transform(X_test_low))

print("Правильность на 75% тренировочного набора: {:.2f}".format(score_low))
print("Accuracy: {:.3f}".format(accuracy_score(y_pred_low, y_test_low)))
# Правильность на 75% тренировочного набора: 0.97
# Accuracy: 0.949

In [ ]:
# обучим тренировочный набор на данных с категорией по средней иерархии
X_train_mid, X_test_mid, y_train_mid, y_test_mid = train_test_split(combined_data_frame['title_desc_price'], 
                                                                    mid_level,
                                                                    shuffle=False)
# обучение модели
model_mid = LogisticRegression(C=0.01).fit(X_data_vect, y_train_mid)
score_mid = model_mid.score(X_data_vect, y_train_mid)
y_pred_mid = model_mid.predict(vect.transform(X_test_mid))

print("Правильность на 75% тренировочного набора: {:.2f}".format(score_mid))
print("Accuracy: {:.3f}".format(accuracy_score(y_pred_mid, y_test_mid)))
# Правильность на 75% тренировочного набора: 0.96
# Accuracy: 0.928

In [ ]:
# обучим тренировочный набор на данных с категорией по средней иерархии
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(combined_data_frame['title_desc_price'], 
                                                            hight_level,
                                                            shuffle=False)
# обучение модели
model_h = LogisticRegression(C=0.01).fit(X_data_vect, y_train_h)
score_h = model_h.score(X_data_vect, y_train_h)
y_pred_h = model_h.predict(vect.transform(X_test_h))

print("Правильность на 75% тренировочного набора: {:.2f}".format(score_h))
print("Accuracy: {:.3f}".format(accuracy_score(y_pred_h, y_test_h)))
# Правильность на 75% тренировочного набора: 0.93
# Accuracy: 0.87

In [ ]:
# прочитаем и применим функции очистки и объядинения для тестового набора
data_frame_test = pd.read_csv('test.csv')
#data_frame_test.info() # пустых значений в датафрейме нет

In [ ]:
# применим функцию clean_corpus к столбцам датафрейма
data_frame_test['description'] = data_frame_test['description'].apply(clean_corpus)
data_frame_test['title'] = data_frame_test['title'].apply(clean_corpus, num=2)
# объединим заголовок, описание и цену в одну колонку
combined_data_test = combine_columns(data_frame_test, 'title_desc', 
                                      'title', 'description')
combined_data_test = combine_columns(combined_data_test, 'title_desc_price', 
                                      'title_desc', 'price')
# после объединения необходимо повторно удалить дубликаты из столбца
# так как заголовок вместе с описанием могут содержать их 
combined_data_test['title_desc_price'] = combined_data_test['title_desc_price'].apply(clean_data)
# предскажем категории для тестовых данных
X_test_new = combined_data_test['title_desc_price']
test_predictions = model.predict(vect.transform(X_test_new))

In [ ]:
# запишем в 'result_for_test.csv' предсказанные категории и id объявлений
result_test_frame = pd.DataFrame({'category_id':test_predictions, 'item_id':data_frame_test['item_id']})
result_test_frame.to_csv('result_for_test.csv')
# p.s. при обучении набора на 100% тренировочных данных,
# категории будут определены точнее
# при визуальном анализе объявлений и предсказанных категорий 
# практически все кажется верным. Можно уотверждать правильность ~87-88 согласно accuracy%.